# About this document

The purpose of this noted version of pipeline is to guide the user through each step of the codes so that they are able to understand them as much as possible and equipped with knowledge to customize the script regardless of programming skills. It is **not** supposed to be runned as production tool. If you click "Run All" on this document, you **will** encounter errors. For productive purpose, consider modifying the accompanying **pipeline.ipynb** to suit your need.

Before we start, it's highly recommended that you get familiar with basic python concepts and operations like [string manipulation](https://docs.python.org/3.4/library/string.html), [tuples, lists and dictionaries](https://docs.python.org/3/tutorial/datastructures.html), as well as a little bit about [object-oriented programming](https://python.swaroopch.com/oop.html) and [python modules](https://docs.python.org/3/tutorial/modules.html).

Another note on the styling of this document: most of the sentences should (hopefully) make sense if taken literally. However, I try to use some special formatting on the texts consistently to demonstrate the close relationship between the codes and thought process, as well as encouraging the reader to understand the already natural and self-explantory Python syntax and jargons. Specifically:

-  a [hyperlink](https://en.wikipedia.org/wiki/Hyperlink) usually point to a well-defined python module or class or methods, especially when that concept is first encountered in this document and the reader would likely need some background reference. The link usually poing to the official documentation of that concept, which might not be the best place to start for beginner. If you find the documentation puzzling, try to google that concept and find a tutorial that best suit your preference.
-  an inline `code` usually refer to a name that already exsist in the [namespace](https://docs.python.org/3/tutorial/classes.html#python-scopes-and-namespaces) (i.e. the context where we run the codes in this document). It can be a previously encountered concepts, but more often it referes to variable or method names that we [imported](https://docs.python.org/3/reference/import.html) or defined along the way.
-  **bold** texts are used more loosely to highlight anything that requires more attention than plain texts. Though it's not used as carefully as previous formats, it often refer to some specific values that a variable or method arguments can assume.

# Pipeline
## Setting up
### set module paths and data path

Ideally, the following cell would be the only thing you have to change when analyzing different dataset. `minian_path` should be the path that contains a folder named **"minian"** with actual python codes (.py files) under it. Similarly `caiman_path` should contain a folder named **"caiman"** with codes. `dpath` is the folder that contains the actual videos, which are usually named **"msCam\*.avi"** where \* is a number. `meta_dict` is a python [dictionary](https://docs.python.org/3/tutorial/datastructures.html#dictionaries) that is used to construct meta data for the final labeled data structure. Currently we assume data are stored in some structured folders and we use folder names to fill in information. The default value means: the name of the last folder in `dpath` (the one that directly contains the videos) would be used as the value of a field named **'session_id'**, the name of the second-last folder in `dpath` would be the value for **'session'** and so on. Both the `keys` (field names) and `values` (numbers indicating which level of folder name should be used) of `meta_dict` can be modified to suit the specific user case.

In [ ]:
minian_path = "."
caiman_path = "."
dpath = "./demo_movies/"
meta_dict={'session_id': -1, 'session': -2, 'animal': -3}

### load modules

This cell loads all the modules required. Usually should not be modified, and should be executed every time the kernel is restarted.

In [ ]:
%%capture
%load_ext autoreload
%autoreload 2
import sys
import os
sys.path.append(minian_path)
sys.path.append(caiman_path)
import gc
import psutil
import numpy as np
import xarray as xr
import holoviews as hv
import functools as fct
import paramnb
import caiman as cm
import matplotlib.pyplot as plt
import bokeh.plotting as bpl
import dask.array as da
import minian.visualization_ply as mvis
from bokeh.io import output_notebook, show
from bokeh.layouts import layout
from bokeh.application import Application
from bokeh.application.handlers import FunctionHandler
from IPython.core.display import display, HTML
from ipyparallel import Client
from minian.utilities import load_videos, varray_to_tif, save_cnmf, save_movies, scale_varr, save_varr
from minian.preprocessing import remove_background, detect_brightspot, correct_brightspot, gaussian_blur
from minian.motion_correction import estimate_shift_fft, apply_shifts, interpolate_frame
from minian.visualization import VArrayViewer, MCViewer, CNMFViewer
from minian.caiman_patch import local_correlations_fft, correlation_pnr
from caiman.cluster import setup_cluster
from caiman.source_extraction import cnmf
from caiman.utils.visualization import inspect_correlation_pnr, nb_view_patches
from caiman.components_evaluation import estimate_components_quality_auto

### module initialization

Some further initialization steps to run. Usually should not be modified, and should be executed every time the kernel is restarted.

In [ ]:
cm.summary_images.local_correlations_fft = local_correlations_fft
cm.summary_images.correlation_pnr = correlation_pnr
dpath = os.path.abspath(dpath)
hv.notebook_extension('bokeh', width=100)

## Pre-processing
### loading videos and visualization
The first argument of `load_videos` should be the path that contains the videos, which could be `dpath` we already defined. The second argument `pattern` is optional and is the [regular expression](https://docs.python.org/3/library/re.html) used to filter files under the specified folder. The default value `'msCam[0-9]+\.avi$'` means that a file can only be loaded if its filename contain **'msCam'** then followed by at least one digit of number then **'.avi'** as the end of the filename. This can be changed to suit the naming convention of your videos.

In [ ]:
varr = load_videos(dpath, pattern='msCam[0-9]+\.avi$')

The previous cell load videos and concatenate them together into `varr`, which is a [xarray.DataArray](http://xarray.pydata.org/en/stable/generated/xarray.DataArray.html#xarray.DataArray). Now it's a perfect time to get familiar with this data structure and the [xarray](https://xarray.pydata.org/en/stable/) module in general, since we will be using these data structures for most of the pre-processing steps as well as the final output of our analysis. Basicly a `xarray.DataArray` is a labeled N-dimensional array. We can ask the computer to print out some information of `varr` by calling its name (as with any other variable):

In [ ]:
varr

We can see now that `varr` is a `xarray.DataArray` with a [name](https://xarray.pydata.org/en/stable/generated/xarray.DataArray.name.html#xarray.DataArray.name) `'demo_movies'` and three dimensions: `frame`, `height` and `width`, each dimension is simply labeled with ascending natural numbers. The [dtype](https://xarray.pydata.org/en/stable/generated/xarray.DataArray.dtype.html#xarray.DataArray.dtype) ([data type](https://docs.scipy.org/doc/numpy-1.14.0/user/basics.types.html)) of `varr` is `numpy.uint8`

In addition to these information, we can visualize `varr` with the help of `VArrayViewer`, which shows the array as movie and also plot max, mean and minimum fluorescence:

In [ ]:
%%output size=100 fps=30
vaviewer = VArrayViewer([varr], framerate=30)
display(vaviewer.widgets)
vaviewer.show()

Before proceeding to pre-processing, it's good practice to check if there is something obviously wrong with the video (like camera suddenly drops dark). This can usually be observed by visualizing the video and check the mean fluorescence plot. To take out some bad `frame`, let's say, `frame` after 800, we can utilize the [xarray.DataArray.sel](http://xarray.pydata.org/en/stable/generated/xarray.DataArray.sel.html) method and [slice](https://docs.python.org/3/library/functions.html#slice):

In [ ]:
varr = varr.sel(frame=slice(None, 800))

This will subset `varr` along the `frame` dimension from the begining to the `frame` labeled **800**, and assign the result back to `varr`, which is equivalent to taking out `frame` from **801** to the end. Note you can do the same thing to other dimensions like `height` and `width` to take out certain pixels of your video for all the `frame`s. For more information on using `xarray.DataArray.sel` as well as other indexing strategies, see [xarray documentation](http://xarray.pydata.org/en/stable/indexing.html)

### background removal

The `remove_background` method takes two arguments: first is the video-array we want to operate on, which is `varr`. The second `window` argument is optional and defaulted to **51**. Under the hood this method pass each `frame` of the array to a [uniform filter](https://docs.scipy.org/doc/scipy/reference/generated/scipy.ndimage.uniform_filter.html) to get an estimation of **"background"**, then this **"background"** is subtracted from each `frame` to give the result. The `window` parameter controls the window size of the filter, *i.e.* the `size` parameter in [scipy.ndimage.uniform_filter](https://docs.scipy.org/doc/scipy/reference/generated/scipy.ndimage.uniform_filter.html). The result is assigned to a new variable `varr_ft` (stands for video-array filtered). Of course you can assign it to whatever name you like, even back to `varr` if you are sure you don't want to visualize the original `varr` anymore and want to save some memory.

In [ ]:
varr_ft = remove_background(varr, window=51)

### bright spots removal

A lot of times the camera capture dusts on the lense and you can see some persistent "bright spots" in the videos. The following cell try to correct these artifacts. If your video is nice and clean, feel free to skip this step to minimize arbituary manipulation of raw data.

Since the bright spots are usually persistent across `frame`s, the `detect_brightspot` method first try to find them on the **mean frame** of the video array (*i.e.* `varr_ft.mean('frame')`, see [xarray.DataArray.mean](http://xarray.pydata.org/en/stable/generated/xarray.DataArray.mean.html) for details). It inspect the **mean frame** with square rolling windows, and pick out bright (large positive) outliers within each window according to the [zscore](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.zscore.html) of each pixels in that window. `detect_brightspot` can take in four arguments: first one is the array we want to operate on as usuall, and we use `varr_ft` from last step; `thres` is optional and defaulted to `None`, it's the threshold of zscore that we use to pickout outliers. If it's `None` then the script will use the inverse of the smallest negative zscore in each window as `thres`. However pragmatically setting it to **4** tend to give cleaner estimation; `window` is optional and defaulted to **50**, it is the size of the rolling windows we use in pixel; Finally `step` is also optional and defaulted to **10**, specifying the distance between the center of each rolling window in pixel, in other words, how closely the overlapping windows should be placed. The result of `detect_brightspot` is assigned to `spots`, which is also a `xarray.DataArray` specifying how many times each pixel is identified as "bright spots" summed across all windows. The idea is that a bright spot should be identified as outliers idependently in many windows that include that spot, thus having a large number in `spots`, with a theoratical maximum possible value equal to `(window / step)**2`.

Then, `correct_brightspot` is used to correct the `spots`. It achieve so by interpolating the `spots` with the mean of other pixels surrounding the spot within a `window`. It can take five arguments: first is the array to operate on; second is an array specifying where are the `spots`; `window` is the size of window used for interpolation in pixels; `spot_thres` specify a threshold for numbers in `spots` above which a pixel will be considered as an actuall bright spot and corrected. If you use the default `window` and `step` in `detect_brightspot`, a `spot_thres` of **10** works well pragmatically. Lastly `inplace` is optional and defaulted to `True`, it determined whether the correction should be carried out in-place. Feel free to leave it to `True` if you don't need to compare the result before and after the bright spot corrections. The behavior of `window` and `spot_thres` also depends on how many dimensions `spots` has (*i.e.* `len(spots.dims)`) - if `spots` has only two dimensions (as in the case where `spots` is the result from `detect_brightspot`), `window` will be a two-dimensional window and `spot_thres` will be used. In this case the mean will be taken across all `frame`s during interpolation. However, if `spots` has three dimensions, `window` will also be three-dimensional and `spot_thres` will be automatically set to **0**, so that setting `spot_thres` not longer has any effect. In this case the interpolation is carried out with only the pixels neighbouring to the spot in `height`, `width` and `frame`.

In [ ]:
spots = detect_brightspot(varr_ft, thres=4, window=50, step=10)
varr_ft_ds = correct_brightspot(varr_ft, spots, window=2, spot_thres=10, inplace=False)

Some times a bright spot can also appear only on a few frames, escaping the previous **mean frame**-based methods. The following cell try to correct that by simply taking out and interpolate the largest **0.0000001** pixels in three dimensions. Again skip this if no such bright spots are observed for your videos.

The [xarray.DataArray.quantile](http://xarray.pydata.org/en/stable/generated/xarray.DataArray.quantile.html) method calculate the **0.9999999** quantile for the whole data, then a vectorized comparison `>` produce a boolean mask specifying where are the largest `outliers`, and then `outliers` can be used with `correct_brightspot` as described before. The result is assigned to `varr_ref` standing for "video-array reference". This step is completely arbituary and you can modify it to better suit your data. Just beware that using a smaller level of quantile will result in huge number of pixels being treated as outliers.

In [ ]:
outliers = varr_ft_ds > varr_ft_ds.quantile(0.9999999)
varr_ref = correct_brightspot(varr_ft_ds, outliers)

### gaussian blur

This step simply apply a [gaussian filter](https://docs.opencv.org/3.3.0/d4/d86/group__imgproc__filter.html#gaabe8c836e97159a9193fb0b11ac52cf1) to `varr_ref` and assign the result back to `varr_ref`. This is required for motion correction to work well on Miniscope data. Both `ksize` and `sigmaX` are optional and passed through to [cv2.GaussianBlur](https://docs.opencv.org/3.3.0/d4/d86/group__imgproc__filter.html#gaabe8c836e97159a9193fb0b11ac52cf1). Please refer to the documentation for more detail.

In [ ]:
varr_ref = gaussian_blur(varr_ref, ksize=(3, 3), sigmaX=0)

### normalization

This final step of pre-processing normalize the result to the range of \[0, 255\] (inclusive), and then convert it to `numpy.uint8` data type. The `scale` argument of `scale_varr` should be a [tuple](https://docs.python.org/3/tutorial/datastructures.html#tuples-and-sequences). It's optional and defaulted to `(0, 1)` meaning by default it will normalize the input to the range of \[0, 1\]. The `copy=False` in [xarray.DataArray.astype](http://xarray.pydata.org/en/stable/generated/xarray.DataArray.astype.html) basically means operating in-place. Refer to the documentation of `xarray.DataArray.astype` for more detail.

In [ ]:
varr_ref = scale_varr(varr_ref, scale=(0, 255)).astype(np.uint8, copy=False)

### visualization of pre-processing

Finally the visualization of the pre-processing results. Note that you can feed in a [list](https://docs.python.org/3/tutorial/introduction.html#lists) of video-arrays to `VArrayViewer` to visualize them. Also you can use the `%%output size=55` cell magic [to customize the global scaling](http://holoviews.org/user_guide/Customizing_Plots.html) of all the plots to fit your screen. The following cell visualize the original video, the background-removed video and the final pre-processed video side by side.

In [ ]:
%%output size=55 fps=30
vaviewer = VArrayViewer([varr, varr_ft, varr_ref], framerate=30)
display(vaviewer.widgets)
vaviewer.show()

## motion correction
### estimate shifts

The `estimate_shift_fft` method estimates shifts of each frame by calculating a cross-correlation of each frame with a common template using [fft](https://en.wikipedia.org/wiki/Fast_Fourier_transform), and then the centroids of the cross-correlation is found using [scipy.ndimage.center_of_mass](https://docs.scipy.org/doc/scipy/reference/generated/scipy.ndimage.center_of_mass.html) and is returned as the shift of that frame. At the same time the maximum value of the cross-correlation is also returned. The shifts and maximum correlation of each frame is concatenated together as `shifts` and `corr`. After this, a z-score is taken for `corr`, and any frame that has a z-score lower than `z_thres` is treated as a "bad" frame, usually because a frame shifts so dramatically that cannot be registered accurately. Finally a `mask` is returned to specify which frames are "bad".

`estimate_shift_fft` can take in four arguments: the first is video-array; `z_thres` is the z-score threshold used to filter out "bad" frames. If your video doesn't have a dramatic shift, leave it to `None` so that no thresholding will be performed. `on` indicate which frame should be used as template. It can be `'first'`, `'last'` or `'mean'`. By default `'first'` frame will be used. `pad_f` can be used to pad the frames with extra **0**s to get rid of edge effects of fft operation, however usually leaving it to **1** (no padding) works fine.

In [ ]:
shifts, corr, mask = estimate_shift_fft(varr_ref, z_thres=None, on='first', pad_f=1)

### apply shifts

`apply_shifts` will apply the `shifts` from last step to the supplied `varr_ref`. The resulting array is assigned to `varr_mc`, and `shifts_final` should be the same with `shifts`

In [ ]:
varr_mc, shifts_final = apply_shifts(varr_ref, shifts)

### interpolation

If you used a `z_thres` in `estimate_shift_fft` and thus having a meaningful `mask`, you can use `interpolate_frame` to interpolate those "bad" frames in `mask` by simply taking a mean of the two neighbouring frames. Else just skip this step.

In [ ]:
varr_mc_int = interpolate_frame(varr_mc, mask)

### visualization of motion-correction

Visualization of the motion-correction results. The three movies are the original one, the motion-corrected one, and the motion-corrected and interpolated one. If you skipped the interpolation step, remember to take out `varr_mc_int` since it's not defined.

In [ ]:
%%output size=55 fps=10
vaviewer = VArrayViewer([varr, varr_mc, varr_mc_int], framerate=10)
display(vaviewer.widgets)
vaviewer.show()

### visualization of shifts

Visualization of shifts. They should be relatively small and centered around zero. If not, use the visualization tools above to check whether the shifts are real.

In [ ]:
%%output size=100
%%opts Curve [width=1500, tools=['hover']]
hv.NdOverlay(dict(width=hv.Curve(shifts.sel(shift_dim='width')), height=hv.Curve(shifts.sel(shift_dim='height'))))

### save result as tif

Save the video array as tif for CaImAn to read in. The first argument to `varray_to_tif` is the path of the file to write, and the second argument is the array to be converted and saved. We will save a file named "varr_mc_int.tif" under `dpath`. FYI [os.sep](https://docs.python.org/3/library/os.html#os.sep) is simply an [OS](https://en.wikipedia.org/wiki/Operating_system)-specific path separator, and `dpath + os.sep + "varr_mc_int.tif"` is simply a string representing the full path of the .tif file.

In [ ]:
varray_to_tif(dpath + os.sep + "varr_mc_int.tif", varr_mc_int)

### save result as DataSet

Save the video as [xarray.Dataset](http://xarray.pydata.org/en/stable/generated/xarray.Dataset.html), with proper meta-data populated. The first argument of `save_varr` is the array to save, the second argument is the folder to put the file in. The file will be named "varr_mc_int.nc" regardless of the arguments passed in. `meta_dict` is defined at the begining of this document to specify how the information of meta-data can be obtained from folder names.

In [ ]:
varr = save_varr(varr_mc_int, dpath, meta_dict=meta_dict)

We can see that `varr` is now a `xarray.Dataset` with additional coordinates corresponding to meta-data.

In [ ]:
varr

## CNMF

Before we go on to CNMF, it's a good time now to restart the kernel to free up some memory, since all the results in previous steps are saved in hard drive and there are a few large intermediate arrays that we don't need any more.

### save data as memmap

In order for CaImAn to function properly and efficiently, the data has to be converted to [numpy memory-map](https://docs.scipy.org/doc/numpy/reference/generated/numpy.memmap.html) format. The first argument to `cm.save_memmap` is a `list` of filenames to be converted. We'll use the .tif file we just saved. `base_name` serves like a perfix to the filenames of the resulting memory-map files. `order='F'` is required to work flawlessly with CaImAn.

In [ ]:
fname = cm.save_memmap([dpath + os.sep + "varr_mc_int.tif"], base_name='varr_mc_int', order='F')
# fname = "./demo_movies/varr_mc_int_d1_480_d2_752_d3_1_order_F_frames_1000_.mmap"

### set up and load data

Starting up the parallel workers pool for CaImAn. An [Ipython Direct View](http://ipyparallel.readthedocs.io/en/latest/direct.html?highlight=direct%20view) object is returned as `dview` and needed to be passed in to other CaImAn functions. If you don't want to utilize parallel computation, skip this step and pass in `dview=None` for all functions below that accept a `dview` argument. Some times leaving the parallel pool idle for a while makes it no longer functional. In this case just re-run the following cell to shutdown (`dview.terminate()`) and restart (`setup_cluster`) the parallel pool.

In [ ]:
os.chdir(caiman_path)
try:
    dview.terminate() # stop it if it was running
except:
    pass
c, dview, n_processes = setup_cluster(
    backend='local', # use this one
    n_processes=16,  # number of process to use, if you go out of memory try to reduce this one
)

The following cell load in the data from memory map and reshape it back to 3 dimensions (since arrays are saved flattened in memory map)

In [ ]:
Yr, dims, T = cm.load_memmap(fname)
Y = Yr.T.reshape((T,) + dims, order='F')

### cnmf update

This is the step that actually run the CNMF algorithm. Basicly it creates a `caiman.CNMF` object named `cnm` with a set of parameters, and just fit the videos to them by `cnm.fit(Y)`. This will usually take a while and tweaking parameters in this step will have most impacts for the result. The following parameters worked well for CA1 data. Refer to CaImAn documentation for more information.

In [ ]:
cnm = cnmf.CNMF(n_processes=n_processes, 
                method_init='greedy_roi',               # use this for 1 photon
                k=15,                                   # neurons per patch
                gSig=(5, 5),                            # half size of neuron
                merge_thresh=.8,                        # threshold for merging
                p=1,                                    # order of autoregressive process to fit
                dview=dview,                            # if None it will run on a single thread
                tsub=2,                                 # downsampling factor in time for initialization, increase if you have memory problems             
                ssub=2,                                 # downsampling factor in space for initialization, increase if you have memory problems
                Ain=None,                               # if you want to initialize with some preselcted components you can pass them here as boolean vectors
                rf=(60, 60),                            # half size of the patch (final patch will be 100x100)
                stride=(40, 40),                        # overlap among patches (keep it at least large as 4 times the neuron size)
                only_init_patch=True,                   # just leave it as is
                gnb=1,                                  # number of background components
                method_deconvolution='oasis',           # could use 'cvxpy' alternatively
                low_rank_background=True,               # leave as is
                update_background_components=True,      # sometimes setting to False improve the results
                normalize_init=False,                   # just leave as is
                center_psf=True,                        # leave as is for 1 photon
                del_duplicates=True)                    # whether to remove duplicates from initialization

cnm.fit(Y)

### components evaluation

`estimate_components_quality_auto` will try to reject some trash units for you. The arguments that has most impacts are `min_SNR`, `r_values_min` and `min_std_reject`. The result will be returned as the index of good components - `idx_components` and the index of bad components - `idx_components_bad`, as well as some variables that are thresholded during the process. Refer to CaImAn documentation for more information.

In [ ]:
idx_components, idx_components_bad, comp_SNR, r_values, pred_CNN = estimate_components_quality_auto(
                            Y, cnm.A, cnm.C, cnm.b, cnm.f, cnm.YrA, 30, 
                            0.4, cnm.gSig, dims, dview = dview, 
                            min_SNR=2, r_values_min = 0.8, min_std_reject = 1, use_cnn = False)
dview.terminate()

### save results

Save the results as `xarray.DataSet` and [python pickle](https://docs.python.org/3/library/pickle.html) files. The first argument to `save_cnmf` is the `caiman.CNMF` object to save; The second argument is the folder to put files in. `unit_mask` is a mask of accepted units. We'll use the `idx_components` from last step. `meta_dict` indicate how meta data are populated, and `order='F'` ensure the consistency during reshaping.

In [ ]:
try:
    cnmfds.close()
except NameError:
    pass
cnmfds = save_cnmf(cnm, dpath, unit_mask=idx_components, meta_dict=meta_dict, order='F')

### visualization

Finally, we can load the .nc files we saved before with [xarray.open_dataset](http://xarray.pydata.org/en/stable/generated/xarray.open_dataset.html) and [xarray.open_dataarray](http://xarray.pydata.org/en/stable/generated/xarray.open_dataarray.html), and feed them into `CNMFViewer` to visualize them. The way `CNMFViewer` functions is a bit counter-intuitive - a server listening to `port` will be runned on the machine that calls `CNMFViewer.show`, and the server will be responsible to handle all the interactions for `CNMFViewer`. The cell that runs `CNMFViewer.show` will never finish by itself untill you explicitly interrupt it which will kill the server, so be sure to do that after you are done with visualization otherwise the whole document will stuck. If you are visiting this document with [http](https://en.wikipedia.org/wiki/Hypertext_Transfer_Protocol) connection, the visualization should show below as the output of the cell that runs `CNMFViewer.show`. However if you are visiting this document with [https](https://en.wikipedia.org/wiki/HTTPS) connection, by default you could only see an empty frame there since the visualization scripts are not encrypted. There are two ways you can access it - either find a way to ask your browser to temporally allow for unsecure contents on this page and refresh (potentially re-run the `CNMFViewer.show` line), or go to `http://[hostname]:[port]` in your browser where `[hostname]` is the [hostname](https://en.wikipedia.org/wiki/Hostname) of the machine you are running this notebook (should be `localhost` if you are running locally on your PC), and `[port]` is the number you feed to the `port` argument of `CNMFViewer.show`.

In [ ]:
cnmfds = xr.open_dataset(dpath + os.sep + "cnm.nc")
Yds = xr.open_dataarray(dpath + os.sep + "varr_mc_int.nc")
cnmfviewer = mvis.CNMFViewer(cnmfds, Yds)

In [ ]:
cnmfviewer.show(port=10000)